## Clean and concactenate all the pathology reports into one file

In [1]:
import numpy as np
import cPickle
from collections import defaultdict
from collections import Counter
import sys, re, os
import pandas as pd
import random,pprint
from sklearn.cross_validation import StratifiedKFold
homePath = os.path.expanduser("~")
pp = pprint.PrettyPrinter(indent = 4)

In [2]:
def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

def clean_str_sst(string):
    """
    Tokenization/string cleaning for the SST dataset
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

def cleanReadReport(infile,fileDir):
    #stage 1: clean document by line
    reportPath=os.path.normpath(fileDir+'/'+infile)
    outfile=""
    with open(os.path.normpath(reportPath), 'r') as inFile:
        readFile = inFile.read()
    cleanedFile = cleanDoc(readFile)
    return cleanedFile

def cleanDoc(inDoc):
    outDoc=[]
    #print '==========================================='
    #print 'inline ', inDoc
    #print '==========================================='
    for line in inDoc.splitlines():
        outDoc.append(cleanLine(line.strip()))
        output=' '.join(outDoc)
        #print 'before', output
        tokenized=[cleanWord(word) for word in output.split(" ") \
            if len(cleanWord(word))>0]
		#print 'after', tokenized
    #thisText=Text(' '.join(tokenized))
    #print '-------------------------------------------'
    #print 'outline'
    #pp.pprint(thisText.sentences)
    #print '-------------------------------------------'
    return tokenized

reWord=re.compile(r'[^a-zA-Z0-9]')
#sub7=re.compile(r'[0-9]{1,}[\.]{0,1}[0-9]{0,1}')
def cleanWord(text):
    text = text.strip()
    #text= sub7.sub('NUMBER',text)
    return text

sub1=re.compile(r'<.*?>')
sub2=re.compile(r'\\{1,2}X[a-zA-Z0-9]{1,}\\?')
#sub3=re.compile(r'[**]\/?[^\]]*]')
sub4=re.compile(r'([^0-9\s])\1{2,}')
#sub5=re.compile(r'[[]\/?[^>]*[]]')
sub6=re.compile(r'[A-Z][a-z]+[A-Z][a-z]+[:]')

def cleanLine(text):
    text = sub1.sub(' ',text)
    text = sub2.sub(' ',text)
    #text = sub3.sub('',text)
    text = sub4.sub(' ',text)
    #text = sub5.sub('',text)
    text = sub6.sub(' ',text)
    text= reWord.sub(' ',text)
    return text

In [3]:
usingDir='/Dropbox/pathology'

In [4]:
#get each class by dir
classDirs=[name for name in os.listdir(os.path.normpath(homePath+usingDir)) \
            if os.path.isdir(os.path.normpath(homePath+usingDir+'/'+name))]
classDirs.sort()

print classDirs

['c34.0', 'c34.1', 'c34.2', 'c34.3', 'c34.9', 'c50.1', 'c50.2', 'c50.3', 'c50.4', 'c50.5', 'c50.8', 'c50.9']


In [5]:
# get the path of each files
fileByClass=[os.listdir(os.path.normpath(homePath+usingDir+'/'+thisClass))
             for thisClass in classDirs]

### Load and clean all the files

In [6]:
# Read all the files
revs = []
classes = []
lenCompare = []
lenDict = {}
for i in range(len(classDirs)):
    for report in fileByClass[i]:
        if report.endswith('.txt'):
            thisReportDir = os.path.normpath(homePath+usingDir+'/'+classDirs[i])
            cleanedReport = cleanReadReport(report, thisReportDir)
            text = " ".join(cleanedReport).strip()
            revs.append(text)      

### Output file 'raw_text_upper.txt'

In [8]:
# write all the files into one txt file
with open('raw_text_upper.txt', 'w') as f:
    for report in revs:
        print >> f, report